This notebook will be used to complete the Data Science Capstone project for Coursera, and serve as a more detailed description and "deep dive" on methods used to pull, clean, and analyze the Foursquare data.

In [1]:
import pandas as pd
import numpy as np

# More needed dependencies, taken from Week 3 lab on NYC:
import requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

import json

import pickle # for saving pulled foursquare data later

print('Libraries imported.')

Libraries imported.


In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Section 1: Data

### 1.1: Foursquare (Venues)

We will source our venues data from Foursquare. Specifically, we'll aim to get all bars & lounges in New York City. Howver, we have a bit of a snag to work out: Foursquare limits us to only 50 results from a single search query, so we will have to get a little creative to get a sizable dataset to work with. I propose the following:

1. A handy-dandy New York City neighborhoods dataset was provided to us in the "Segmenting and Clustering Neighborhoods in New York City" lab. It included the borough, name, latitude and longitude of all 306 neighborhoods in NYC. If we take the last two bits, the latitude and longitude, we can get Foursquare results for *each* of these neighborhoods.

2. We'll do the same search query on each of these lat/long pairs: 50 results from 306 pairs, which is *15,300 results*. We will store each resulting dataframe into a list.

3. Pandas is magical, so there's obviously a function to combine these into one big dataframe: pd.concat. 

4. However, these results will **not all be unique**, so we'll have to drop all duplicate rows. Again, pandas is magical, so of course there's a simple method for this: df.drop_duplicates().

5. Our only other constraint on the results is that they're located *in* New York City, which may be a little silly (consider a bar right outside the city limits next to 10 others right inside of the city limits), but we'll roll with it for the sake of the product. This is, again, easy to do with pandas.

Let's do it!

#### Step 1: New York City Neighborhoods Dataset

In [3]:
# Much of this code is from the lab, but adapted to use requests instead of wget.

r = requests.get('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json')
newyork_data = r.json()
neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [4]:
# Double check
neighborhoods.shape

(306, 4)

We got it!

#### Step 2: Search Query on Each Lat/Long Pair

Now, we just need to run a regular Foursquare search query on each one of these lat/long pairs. We can iterate over the dataframe; do a search on each row's coordinates; then store the resulting dataframe into a list of results.

**!!! WARNING !!! If you plan on running this yourself, be prepared to make _306_ Foursquare API calls. Remember that a free account without a verified credit card only gets 950 per day.**

In [5]:
CLIENT_ID = 'JCLRUISTSH4ETI2F1GQRNHLJVAWMOACMQVEZJJ2XP2LKVD5U'
CLIENT_SECRET = 'FLX1ZUZ1ZHANYJCTLQ5SWES2IRK3I2UHVBUUX1EQR2XNRELE'
VERSION = '20210121'
QUERY = 'bar'
LIMIT = 50 # max
# The following categoryID will limit results to the 'Nightlife' category, which includes bars; lounges; nightclubs; etc.
# List of category IDs: https://developer.foursquare.com/docs/build-with-foursquare/categories/
CATEGORYID = '4d4b7105d754a06376d81259'


# For later cleaning of search results:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

def clean_results(data):
    # turns data from venues query into a dataframe and cleans it up into an easily-usable format
    results_data = data['response']['venues']
    result = pd.json_normalize(results_data)
    filtered_columns = ['name', 'categories'] + [col for col in result.columns if col.startswith('location.')] + ['id']
    clean_results = result.loc[:, filtered_columns]
    # filter the category for each row
    clean_results['categories'] = clean_results.apply(get_category_type, axis=1)
    # clean column names by keeping only last term
    clean_results.columns = [column.split('.')[-1] for column in clean_results.columns]
    
    return clean_results
    

def fs_venue_batch_query(lldf, CLIENT_ID=CLIENT_ID, CLIENT_SECRET=CLIENT_SECRET, VERSION=VERSION, LIMIT=LIMIT, CATEGORYID=CATEGORYID):
    results = []
    base_uri = 'https://api.foursquare.com/v2/venues/search?'
    
    # Iterate over rows of lat/long dataframe (lldf):
    for index, row in lldf.iterrows():
        LL = '{},{}'.format(row['Latitude'], row['Longitude'])
        payload = {'client_id':CLIENT_ID,
                   'client_secret':CLIENT_SECRET,
                   'll':LL,
                   'v':VERSION,
                   'query':QUERY,
                   'limit':LIMIT,
                   'categoryId': CATEGORYID}
        
        # Pull data with requests
        r = requests.get(base_uri, params=payload)
        data = r.json()

        if r.status_code != requests.codes.ok:
            print('Something went wrong at lat-long {}, {}, index {}. See response:\n'.format(row['Latitude'], row['Longitude'], index))
            json.dumps(data)
            break

        else:
            results.append(clean_results(data))
    
    return results
    

search_results = fs_venue_batch_query(neighborhoods)

In [6]:
print(len(search_results))

306


We got it! Again!

#### Step 3: Combine Into One DataFrame
Simple enough, no?

In [80]:
all_results = pd.concat(search_results)
all_results.reset_index(drop=True, inplace=True)

In [81]:
all_results.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Wine Bar,Wine Bar,at Citi Field,40.756446,-73.846380,"[{'label': 'display', 'lat': 40.75644569750206...",15391,11368,US,Flushing,NY,United States,"[at Citi Field, Flushing, NY 11368, United Sta...",NaN,NaN,4deab02dae60e98923605bc2
1,Last Stop Bar,Bar,4609 White Plains Rd,40.902410,-73.851398,"[{'label': 'display', 'lat': 40.90240976610952...",927,10470,US,Bronx,NY,United States,"[4609 White Plains Rd (east 240th street), Bro...",east 240th street,NaN,5994fb6362420b10800f6429
2,Klassique Bar & Lounge,Bar,3801-3823 U.S. 1,40.881665,-73.838284,"[{'label': 'display', 'lat': 40.88166500000000...",1634,10466,US,Bronx,NY,United States,"[3801-3823 U.S. 1, Bronx, NY 10466, United Sta...",NaN,NaN,4f935dd2e4b00fe65f18aa7c
3,Woodside Inn Bar And Grill,Bar,NaN,40.911785,-73.821836,"[{'label': 'display', 'lat': 40.91178499999999...",2858,NaN,US,Mount Vernon,NY,United States,"[Mount Vernon, NY, United States]",NaN,NaN,4db3432143a1369cb59fa0a5
4,Burke's Bar,Burger Joint,645 Bronx River Rd,40.921848,-73.847107,"[{'label': 'display', 'lat': 40.92184770641938...",3021,10704,US,Yonkers,NY,United States,"[645 Bronx River Rd, Yonkers, NY 10704, United...",NaN,NaN,4e4dd945bd4101d0d79cb2b7


In [82]:
all_results.shape

(15299, 16)

It really was! And we really did get nearly 15,300 results.

**However**, I ran into an issue that I believe is worth mentioning. Notice that I reset the index, and do this repeatedly going forward. This is key to preserving a lot of the results. Without doing this, following the same process, I ended up with less than 200 results. One of these functions along the way gets rid of duplicate indices (not sure which one, but I haven't given that much thought - probably the most likely candidate, drop_duplicates()). If you attempt to do something similar, remember to reset your indices!

Anyway, the data looks great now, so back to the show.

#### Step 4: Drop All Duplicate Venues

Turns out, drop_duplicates doesn't work if some of your values are lists. This is only an issue because of the "labledLatLngs" and "formattedAddress" columns, neither of which we really need, so I'll just get rid of them, and then keep only the unique results.

**NOTE**: This is an opportunity for some leeway. How do we determine if a particular result is unqiue across all of our data? Do all aspects need to match exactly? The pandas drop_duplicates method actually has a "subset" parameter that lets us choose which columns to consider. For our purposes, I'm going to consider only the 'name' and 'address' columns - if a venue has the same name or address as another, it's probably a duplicate. This might get rid of chains (same name, different address), but that's fine - makes for a more 'local' experience, anyway. :-)

In [8]:
all_results_no_lists = all_results.drop(columns=['labeledLatLngs', 'formattedAddress'])
unique_results = all_results_no_lists.drop_duplicates(subset=['name', 'address']) # only consider uniquely-named bars
unique_results.reset_index(drop=True, inplace=True)

In [9]:
unique_results.shape

(742, 14)

In [10]:
unique_results.head()

,name,categories,address,lat,lng,distance,postalCode,cc,city,state,country,crossStreet,neighborhood,id
0,Wine Bar,Wine Bar,at Citi Field,40.756446,-73.846380,15391,11368,US,Flushing,NY,United States,NaN,NaN,4deab02dae60e98923605bc2
1,Last Stop Bar,Bar,4609 White Plains Rd,40.902410,-73.851398,927,10470,US,Bronx,NY,United States,east 240th street,NaN,5994fb6362420b10800f6429
2,Klassique Bar & Lounge,Bar,3801-3823 U.S. 1,40.881665,-73.838284,1634,10466,US,Bronx,NY,United States,NaN,NaN,4f935dd2e4b00fe65f18aa7c
3,Woodside Inn Bar And Grill,Bar,NaN,40.911785,-73.821836,2858,NaN,US,Mount Vernon,NY,United States,NaN,NaN,4db3432143a1369cb59fa0a5
4,Burke's Bar,Burger Joint,645 Bronx River Rd,40.921848,-73.847107,3021,10704,US,Yonkers,NY,United States,NaN,NaN,4e4dd945bd4101d0d79cb2b7


#### Step 5: Ensure All Venues Are In New York

Slight issue: some values in 'city' are the boroughs, and others are the neighborhoods, not just 'New York'. We can't just do a simple drop command on any rows that don't have city == 'New York' anymore. So, we'll have to get a little creative. We can instead make a list of the unique boroughs and neighborhoods out of the Neighborhoods data, add NYC, and then do a drop command on any rows that have a 'city' value that's not in any of the places to include. It's a little complicated, but can be done with just a few lines of code (isn't Python awesome?).

And finally, we'll drop any rows that don't have lat/long values, just in case.

In [21]:
neighborhoods_to_keep = list(neighborhoods.Borough.unique())
neighborhoods_to_keep.extend(list(neighborhoods.Neighborhood.unique()))
neighborhoods_to_keep.extend(['New York', 'New York City'])

ny_bars = unique_results.drop(unique_results[~unique_results.city.isin(neighborhoods_to_keep)].index)
ny_bars.dropna(axis=0,how='any',subset=['lat','lng'],inplace=True)
ny_bars.reset_index(drop=True, inplace=True)

In [22]:
ny_bars.head(20)

,name,categories,address,lat,lng,distance,postalCode,cc,city,state,country,crossStreet,neighborhood,id
0,Wine Bar,Wine Bar,at Citi Field,40.756446,-73.846380,15391,11368,US,Flushing,NY,United States,NaN,NaN,4deab02dae60e98923605bc2
1,Last Stop Bar,Bar,4609 White Plains Rd,40.902410,-73.851398,927,10470,US,Bronx,NY,United States,east 240th street,NaN,5994fb6362420b10800f6429
2,Klassique Bar & Lounge,Bar,3801-3823 U.S. 1,40.881665,-73.838284,1634,10466,US,Bronx,NY,United States,NaN,NaN,4f935dd2e4b00fe65f18aa7c
3,Tommy Bahama's Bar,Sports Bar,Yankee Stadium,40.829045,-73.927661,9965,10452,US,Bronx,NY,United States,"Gate 4, upstairs from Great Hall",NaN,4bf1ec69324cc9b672a5cc92
4,The Liffy II Bar,Bar,5009 Broadway,40.869042,-73.917300,6555,10034,US,New York,NY,United States,NaN,NaN,4b2b1dfbf964a52083b424e3
5,Wegmans Bar,Bar,21 Flushing Ave,40.698325,-73.979432,24537,11205,US,New York,NY,United States,at Nassau St,NaN,5db8c74e5d7a840008f8e09a
6,Inwood Bar and Grill,Restaurant,4892 Broadway,40.867044,-73.922570,7051,10034,US,New York,NY,United States,NaN,NaN,56218101498e22966bef8816
7,BAR-Coastal,Bar,1495 1st Ave,40.771750,-73.953236,16343,10075,US,New York,NY,United States,at 78th St.,NaN,42474900f964a52078201fe3
8,The Grange Bar & Eatery,Cocktail Bar,1635 Amsterdam Ave,40.822554,-73.949532,11778,10031,US,New York,NY,United States,141st St,NaN,51b15855498e85fb32404731
9,Teddy's Bar & Grill,Pub,96 Berry St,40.719205,-73.958431,21668,11249,US,Brooklyn,NY,United States,at N. 8th,NaN,4262f880f964a52025211fe3


In [23]:
print(ny_bars.shape)

(672, 14)


Bingo! We got it. That's our dataset! It's actually become quite slim, down from over 15,000 to just under 650. However, that's more than enough to work with - I was hoping for at least 500.

However, there's a lot of excess information here. Why don't we get rid of most of these unneccesary columns? We'll only keep the name, category, address, latitude, longitude, location, and id - which is probably a bit more than we need, but still a big improvement. I'd like to keep id because, even though it doesn't carry any useful information in and of itself, it is useful as being a unique identifier for each location.

In [27]:
bars = ny_bars.drop(columns=['distance', 'postalCode', 'cc', 'state', 'country', 'crossStreet', 'neighborhood'])
bars.rename(columns={'name':'Name', 
                     'categories':'Category', 
                     'address':'Address', 
                     'lat':'Lat', 
                     'lng':'Lng', 
                     'city':'Location'},
            inplace=True)

bars.head()

,Name,Category,Address,Lat,Lng,Location,id
0,Wine Bar,Wine Bar,at Citi Field,40.756446,-73.846380,Flushing,4deab02dae60e98923605bc2
1,Last Stop Bar,Bar,4609 White Plains Rd,40.902410,-73.851398,Bronx,5994fb6362420b10800f6429
2,Klassique Bar & Lounge,Bar,3801-3823 U.S. 1,40.881665,-73.838284,Bronx,4f935dd2e4b00fe65f18aa7c
3,Tommy Bahama's Bar,Sports Bar,Yankee Stadium,40.829045,-73.927661,Bronx,4bf1ec69324cc9b672a5cc92
4,The Liffy II Bar,Bar,5009 Broadway,40.869042,-73.917300,New York,4b2b1dfbf964a52083b424e3


Perfect. To stop wasting API calls, I'll also save both of these DataFrames with Pickle (the original and the slim versions).

In [28]:
ny_bars.to_pickle('NY Bars, v20210121, Full')
bars.to_pickle('NY Bars, v20210121, Slim')
print('Save successful!')

Save successful!


In [29]:
# Just to make sure:
test_df = pd.read_pickle('NY Bars, v20210121, Slim')
test_df.head()

,Name,Category,Address,Lat,Lng,Location,id
0,Wine Bar,Wine Bar,at Citi Field,40.756446,-73.846380,Flushing,4deab02dae60e98923605bc2
1,Last Stop Bar,Bar,4609 White Plains Rd,40.902410,-73.851398,Bronx,5994fb6362420b10800f6429
2,Klassique Bar & Lounge,Bar,3801-3823 U.S. 1,40.881665,-73.838284,Bronx,4f935dd2e4b00fe65f18aa7c
3,Tommy Bahama's Bar,Sports Bar,Yankee Stadium,40.829045,-73.927661,Bronx,4bf1ec69324cc9b672a5cc92
4,The Liffy II Bar,Bar,5009 Broadway,40.869042,-73.917300,New York,4b2b1dfbf964a52083b424e3


Absolutely amazing. That's our *final* dataset. Really, I won't be making any more changes, promise (at least, in this section).

### 1.2: Folium (Maps)

Now how do these venues look plotted on a map of New York City? Let's find out using Folium. We can use other packages (such as Nominatim) to find its exact latitude and longitude, but since we did that in a previous lab (and the informaiton is readily available through a quick Google search), we'll just hard-code the latitude and longitude: 40.7127281, -74.0060152.

In [19]:
nyc_latitude = 40.7127281
nyc_longitude = -74.0060152

In [30]:
# Now I'm taking code, again, directly from a previous lab (Segmenting and Clustering Neighborhoods in New York City).

# Creating a map of New York City
map_newyork = folium.Map(location=[nyc_latitude, nyc_longitude], zoom_start=10)

# add markers to map
for lat, lng, name in zip(bars['Lat'], bars['Lng'], bars['Name']):
    label = folium.Popup(name, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

Amazing results - we're ready to analyze the data.

## Section 2: Analysis

The algorithm I've chosen to use for this analysis is k-Means, and that's for a couple reasons:

1. It's really simple, easy to understand and modify quickly, and fun to use, in my opinion.
2. The value of k will (can) directly correspond to our "(x) Bar Crawl Locations" list (x = k).

It may not be absolutely perfect for this kind of analysis, but it works well enough, and I'm happy with that.

In [49]:
kClusters = 50 # just to start with
bars_grouped_clustering = bars[['Lat', 'Lng']]
bars_grouped_clustering.head()

,Lat,Lng
0,40.756446,-73.846380
1,40.902410,-73.851398
2,40.881665,-73.838284
3,40.829045,-73.927661
4,40.869042,-73.917300


In [50]:
# run k-means clustering
kmeans = KMeans(n_clusters=kClusters, random_state=0).fit(bars_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([35, 31, 31, 40,  3, 32,  3, 23, 25, 34])

In [51]:
grouped_bars = bars.copy()
grouped_bars.insert(0, 'Cluster Label', kmeans.labels_)
grouped_bars.head()

,Cluster Label,Name,Category,Address,Lat,Lng,Location,id
0,35,Wine Bar,Wine Bar,at Citi Field,40.756446,-73.846380,Flushing,4deab02dae60e98923605bc2
1,31,Last Stop Bar,Bar,4609 White Plains Rd,40.902410,-73.851398,Bronx,5994fb6362420b10800f6429
2,31,Klassique Bar & Lounge,Bar,3801-3823 U.S. 1,40.881665,-73.838284,Bronx,4f935dd2e4b00fe65f18aa7c
3,40,Tommy Bahama's Bar,Sports Bar,Yankee Stadium,40.829045,-73.927661,Bronx,4bf1ec69324cc9b672a5cc92
4,3,The Liffy II Bar,Bar,5009 Broadway,40.869042,-73.917300,New York,4b2b1dfbf964a52083b424e3


Well... that was easy enough. Let's visualize this now using some code again adapted from the "Segmenting and Clustering..." lab.

In [53]:
# create map
map_clusters = folium.Map(location=[nyc_latitude, nyc_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kClusters)
ys = [i + x + (i*x)**2 for i in range(kClusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(grouped_bars['Lat'], grouped_bars['Lng'], grouped_bars['Name'], grouped_bars['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=1,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Wow! That's awesome. If you zoom in and click around, you'll find that even though some colors are really similar, they're actually different destinations. The resolution here is incredible. As expected, the most 'lively' spots are all in and around Manhattan, but there are some other clear clusters well outside the heart of the city.

However...this map is hard to read. There are *fifty* different clusters. It's tough to tell different ones apart where the colors are quite similar, and so the only way to make sure you're looking at the same cluster is to click each one individually and make sure they're the same cluster. Not very efficient.

To address this, let's write up a function to show only one cluster at a time. The clusters aren't ordered in any way, so we'll just write a function to show only one Cluster Label.

In [75]:
def map_cluster(cluster_num, main_lat=nyc_latitude, main_lng=nyc_longitude, data=grouped_bars, k=kClusters):
    #print('Main Lat: {}, Main Lng: {}, k = {}'.format(main_lat, main_lng, k))
    #print(data.head())
    # create map
    main_map = folium.Map(location=[main_lat, main_lng], zoom_start=10)
    cluster_df = data[data['Cluster Label'] == cluster_num]
    

    # add markers to the map
    for lat, lng, name in zip(cluster_df['Lat'], cluster_df['Lng'], cluster_df['Name']):
        label = folium.Popup(name, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=1,
            popup=label,
            color='green',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(main_map)  
        
    return main_map

Let's give it a try!

In [79]:
first_map = map_cluster(2)

first_map

It works! We can show a map of any of the 50 bar crawl locations we found.